In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd


In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# --------------------------
# ⚙️ Cấu hình
# --------------------------
BASE_URL = "https://vnexpress.net/thoi-su"
HEADERS = {"User-Agent": "Mozilla/5.0"}
articles = []

NUM_PAGES = 8  # số trang cần crawl

# --------------------------
# 🕸 Hàm lấy danh sách link bài viết
# --------------------------
def get_article_links(page):
    """
    Lấy danh sách link bài viết từ chuyên mục Chính trị của VNExpress
    """
    url = f"{BASE_URL}/p{page}.html"
    r = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(r.text, "html.parser")
    links = [a["href"] for a in soup.select("h3.title-news a[href]")]
    return list(set(links))  # bỏ trùng

# --------------------------
# 📰 Hàm crawl chi tiết từng bài
# --------------------------
def crawl_article(url):
    """
    Lấy nội dung chi tiết bài viết: title, text, date, source, label
    """
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")

        title_tag = soup.find("h1", class_="title-detail")
        if not title_tag:
            return None
        title = title_tag.get_text(strip=True)

        paragraphs = soup.select("article.fck_detail p.Normal, article.fck_detail p")
        text = " ".join(p.get_text(" ", strip=True) for p in paragraphs)
        if not text:
            return None

        date_tag = soup.find("span", class_="date")
        date = date_tag.get_text(strip=True) if date_tag else ""

        return {
            "title": title,
            "text": text,
            "date": date,
            "source": "vnexpress.net",
            "label": "true"
        }
    except Exception as e:
        print(f"[LỖI] {url} -> {e}")
        return None

# --------------------------
# 🚀 Crawl dữ liệu
# --------------------------
for page in range(1, NUM_PAGES + 1):
    print(f"🔎 Đang crawl trang {page} ...")
    links = get_article_links(page)
    print(f"   → Tìm thấy {len(links)} bài")

    for link in links:
        data = crawl_article(link)
        if data:
            articles.append(data)
        time.sleep(0.5)  # nghỉ tránh bị chặn

    time.sleep(1)

# --------------------------
# 💾 Lưu dữ liệu
# --------------------------
df = pd.DataFrame(articles)
df.drop_duplicates(subset="title", inplace=True)
df.to_csv("vnexpress_true_8pages.csv", index=False, encoding="utf-8-sig")

print(f"\n✅ Crawl xong {len(df)} bài tin thật từ {NUM_PAGES} trang đầu VNExpress.")
print("👉 Dữ liệu lưu tại: vnexpress_true_8pages.csv")
df.head()

🔎 Đang crawl trang 1 ...
   → Tìm thấy 0 bài
🔎 Đang crawl trang 2 ...
   → Tìm thấy 0 bài
🔎 Đang crawl trang 3 ...
   → Tìm thấy 0 bài
🔎 Đang crawl trang 4 ...
   → Tìm thấy 0 bài
🔎 Đang crawl trang 5 ...
   → Tìm thấy 0 bài
🔎 Đang crawl trang 6 ...
   → Tìm thấy 0 bài
🔎 Đang crawl trang 7 ...
   → Tìm thấy 0 bài
🔎 Đang crawl trang 8 ...
   → Tìm thấy 0 bài

✅ Crawl xong 0 bài tin thật từ 8 trang đầu VNExpress.
👉 Dữ liệu lưu tại: vnexpress_true_8pages.csv


""


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# --------------------------
# ⚙️ Cấu hình
# --------------------------
BASE_URL = "https://vnexpress.net/thoi-su-p" 
HEADERS = {"User-Agent": "Mozilla/5.0"}
articles = []

NUM_PAGES = 8  # số trang cần crawl

# 📰 Hàm crawl chi tiết từng bài
# --------------------------
def crawl_article(url):
    """
    Lấy nội dung chi tiết bài viết: title, text, date, source, label
    Chỉ lấy bài có category chứa 'Chính trị'
    """
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")

        # Lấy breadcrumb (dòng định vị chuyên mục)
        breadcrumb = [a.get_text(strip=True) for a in soup.select("ul.breadcrumb a")]
        if not any("Chính trị" in b for b in breadcrumb):
            return None  # bỏ qua bài không thuộc mục Chính trị

        # Lấy tiêu đề
        title_tag = soup.find("h1", class_="title-detail")
        if not title_tag:
            return None
        title = title_tag.get_text(strip=True)

        # Lấy nội dung
        paragraphs = soup.select("article.fck_detail p.Normal, article.fck_detail p")
        text = " ".join(p.get_text(" ", strip=True) for p in paragraphs)
        if not text:
            return None

        # Lấy ngày đăng
        date_tag = soup.find("span", class_="date")
        date = date_tag.get_text(strip=True) if date_tag else ""

        return {
            "title": title,
            "text": text,
            "date": date,
            "category": " > ".join(breadcrumb),
            "source": "vnexpress.net",
            "label": "true"
        }
    except Exception as e:
        print(f"[LỖI] {url} -> {e}")
        return None

# --------------------------
# 🚀 Crawl dữ liệu
# --------------------------
for page in range(1, NUM_PAGES + 1):
    print(f"🔎 Đang crawl trang {page} ...")
    links = get_article_links(page)
    print(f"   → Tìm thấy {len(links)} bài")

    for link in links:
        data = crawl_article(link)
        if data:
            articles.append(data)
        time.sleep(0.5)  # nghỉ tránh bị chặn

    time.sleep(1)

# --------------------------
# 💾 Lưu dữ liệu
# --------------------------
df = pd.DataFrame(articles)
df.drop_duplicates(subset="title", inplace=True)
df.to_csv("vnexpress_chinhtri.csv", index=False, encoding="utf-8-sig")

print(f"\n✅ Crawl xong {len(df)} bài Chính trị từ {NUM_PAGES} trang đầu VNExpress.")
print("👉 Dữ liệu lưu tại: vnexpress_chinhtri.csv")
df.head()


🔎 Đang crawl trang 1 ...
   → Tìm thấy 48 bài


KeyboardInterrupt: 

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# --------------------------
# ⚙️ Cấu hình
# --------------------------
BASE_URL = "https://vnexpress.net/thoi-su-p"   # URL phân trang Thời sự
HEADERS = {"User-Agent": "Mozilla/5.0"}
articles = []

TARGET_COUNT = 500    # số lượng bài cần thu thập
page = 1              # bắt đầu từ trang 1

# --------------------------
# 🕸 Hàm lấy danh sách link bài viết
# --------------------------
def get_article_links(page):
    """
    Lấy danh sách link bài viết từ trang 'Thời sự' của VNExpress
    """
    url = f"{BASE_URL}{page}"
    r = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(r.text, "html.parser")
    links = [a["href"] for a in soup.select("h3.title-news a[href]")]
    return list(set(links))  # bỏ trùng link

# --------------------------
# 📰 Hàm crawl chi tiết từng bài
# --------------------------
def crawl_article(url):
    """
    Lấy nội dung chi tiết bài viết, chỉ giữ bài có nhánh 'Chính trị'
    """
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")

        # --- Kiểm tra breadcrumb có chứa "Chính trị" ---
        breadcrumb = [a.get_text(strip=True) for a in soup.select("ul.breadcrumb a")]
        if not any("Chính trị" in b for b in breadcrumb):
            return None  # bỏ bài không thuộc nhánh Chính trị

        # --- Lấy tiêu đề ---
        title_tag = soup.find("h1", class_="title-detail")
        if not title_tag:
            return None
        title = title_tag.get_text(strip=True)

        # --- Lấy nội dung ---
        paragraphs = soup.select("article.fck_detail p.Normal")
        text = " ".join(p.get_text(" ", strip=True) for p in paragraphs)
        if not text:
            return None

        return {
            "title": title,
            "text": text,
            "url": url,
            "category": " > ".join(breadcrumb),
            "source": "vnexpress.net",
            "label": "true"
        }

    except Exception as e:
        print(f"[LỖI] {url} -> {e}")
        return None

# --------------------------
# 🚀 Vòng lặp crawl chính
# --------------------------
while len(articles) < TARGET_COUNT:
    print(f"🔎 Đang crawl trang {page}... (đã có {len(articles)} bài Chính trị)")
    links = get_article_links(page)
    if not links:
        print(f"❌ Trang {page} không có bài nào, dừng lại.")
        break

    for link in links:
        if len(articles) >= TARGET_COUNT:
            break
        data = crawl_article(link)
        if data:
            articles.append(data)
            print(f"   ✅ {data['title'][:60]}...")
        time.sleep(0.5)  # nghỉ tránh bị chặn

    page += 1
    time.sleep(1)

# --------------------------
# 💾 Lưu dữ liệu
# --------------------------
df = pd.DataFrame(articles)
df.drop_duplicates(subset="url", inplace=True)
df.to_csv("vnexpress_chinhtri_500.csv", index=False, encoding="utf-8-sig")

print(f"\n✅ Crawl hoàn tất! Thu được {len(df)} bài Chính trị.")
print("👉 Lưu tại file: vnexpress_chinhtri_500.csv")


🔎 Đang crawl trang 1... (đã có 0 bài Chính trị)
   ✅ Thủ tướng: Dự báo thiên tai cần sát và kỹ lưỡng hơn...
   ✅ Thường trực Chính phủ yêu cầu nâng cấp hệ thống đê chống lũ...
   ✅ Tổng Bí thư dự lễ khởi công xây trường nội trú ở xã biên giớ...
   ✅ Chủ tịch Hà Nội: Thành phố bị động từ khâu dự báo sau bão Bu...
   ✅ 'Cần làm rõ tính cần thiết của mô hình tập đoàn báo chí'...
   ✅ Ưu tiên chuyển giao công nghệ tạo động lực phát triển đất nư...
   ✅ 'Cần làm rõ tính cần thiết của mô hình tập đoàn báo chí'...
   ✅ Thủ tướng thị sát vùng ngập lụt ven sông ở Hà Nội...
🔎 Đang crawl trang 2... (đã có 8 bài Chính trị)
   ✅ Đề xuất đình chỉ hành nghề bác sĩ tiết lộ giới tính thai nhi...
   ✅ Nhân sự giới thiệu Trung ương khóa 14 được cân nhắc kỹ...
   ✅ Hơn 1.300 tỷ đồng ủng hộ đồng bào vùng lũ và nhân dân Cuba...
   ✅ Hà Nội sẽ đánh giá chỉ số hạnh phúc của người dân...
   ✅ Thủ tướng thị sát vùng lũ Thái Nguyên...
   ✅ Phó thủ tướng: Tư nhân sẽ đảm nhiệm nhiều công trình quan tr...
   ✅ Đề x

In [10]:

# --------------------------
eight_pages = pd.read_csv("vnexpress_true_8pages.csv")
eight_pages_next = pd.read_csv("vnexpress_true_8pagesnext.csv")



# --------------------------
merged_df = pd.concat([eight_pages, eight_pages_next], ignore_index=True)

merged_df.drop_duplicates(subset=["title", "text"], inplace=True)

# --------------------------
# 💾 Lưu file hợp nhất
# --------------------------
merged_df.to_csv("news_merged.csv", index=False, encoding="utf-8-sig")

print(f"\n✅ Gộp xong! Tổng cộng {len(merged_df)} bài.")
print("👉 File đã lưu: newsTrue_merged.csv")

# Xem 5 dòng đầu
merged_df.head()



✅ Gộp xong! Tổng cộng 506 bài.
👉 File đã lưu: newsTrue_merged.csv


,title,text,source,label
0,Thủ tướng: Dự báo thiên tai cần sát và kỹ lưỡn...,"Trưa 9/10, chủ trì cuộc họp Thường trực Chính ...",vnexpress.net,True
1,Việt Nam kêu gọi quốc tế hỗ trợ khắc phục hậu ...,"Chiều 9/10, Bộ Nông nghiệp và Môi trường tổ ch...",vnexpress.net,True
2,"Hà Nội mưa giông 20 phút, một số tuyến phố ngập","Khoảng 8h40, mưa lớn tại khu vực Thanh Xuân, s...",vnexpress.net,True
3,TP HCM tồn đọng gần 100 xe buýt mới khi chuyển...,"Theo báo cáo của Sở Xây dựng TP HCM, số xe trê...",vnexpress.net,True
4,Quân đội chuyển 22.000 chăn màn tới 4 tỉnh ngậ...,Tổng cục Hậu cần Kỹ thuật ngày 10/10 đã điều đ...,vnexpress.net,True


In [9]:
url = 'https://viettan.org/thoi-su/'
response = requests.get(url)
response

ConnectionError: HTTPSConnectionPool(host='viettan.org', port=443): Max retries exceeded with url: /thoi-su/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002A36ADC8550>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
if response.status_code == 200:
    soup = BeautifulSoup(response.content,'html.parser')

In [ ]:
soup.title.text

'Checking your browser - reCAPTCHA'

In [ ]:
# ----------------------------
# Crawl VietTan  after manual CAPTCHA solve
# ----------------------------
import time, random, sys
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ----------------------------
# Config
# ----------------------------
HEADLESS = False          # must be False so user can interact and solve CAPTCHA
# PAGES_TO_CRAWL = [1  50]   # first 2 pages
PAGES_TO_CRAWL = list(range(1, 51))

MIN_ARTICLE_LEN = 200     # filter short articles
USER_AGENT = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
              "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36")

# ----------------------------
# Helper: create selenium driver (headful)
# ----------------------------
def create_driver(headless=HEADLESS):
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument(f"user-agent={USER_AGENT}")
    opts.add_argument("--window-size=1200,900")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    return driver

# ----------------------------
# Heuristic: detect captcha presence in page source
# ----------------------------
def has_captcha(html, driver=None):
    low = (html or "").lower()
    if "recaptcha" in low or "captcha" in low or "verify you are human" in low:
        return True
    try:
        if driver:
            frames = driver.find_elements(By.TAG_NAME, "iframe")
            for f in frames:
                src = (f.get_attribute("src") or "").lower()
                if "recaptcha" in src or "google.com/recaptcha" in src:
                    return True
    except Exception:
        pass
    return False

# ----------------------------
# Transfer selenium cookies -> requests.Session
# ----------------------------
def transfer_cookies_to_requests(driver, session):
    for c in driver.get_cookies():
        session.cookies.set(c['name'], c['value'], domain=c.get('domain'), path=c.get('path'))
    session.headers.update({'User-Agent': USER_AGENT})
    return session

# ----------------------------
# Parse listing page HTML into list of (title, link)
# ----------------------------
def parse_viettan_listing(html):
    soup = BeautifulSoup(html, "lxml")
    items = soup.select("article.jeg_post") or soup.select("article")
    results = []
    for it in items:
        a = it.select_one("h3.jeg_post_title a") or it.select_one("h3 a") or it.select_one("a[href]")
        if not a:
            continue
        title = a.get_text(strip=True)
        link = a.get("href")
        if link and link.startswith("/"):
            link = requests.compat.urljoin("https://viettan.org", link)
        results.append((title, link))
    return results

# ----------------------------
# Fetch article content via Selenium
# ----------------------------
def fetch_article_text_selenium(driver, url):
    try:
        driver.get(url)

        # Đợi bài viết load xong
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "section.elementor-element"))
        )

        time.sleep(2.5)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # -------- LẤY TIÊU ĐỀ (title) --------
        title_section = soup.select_one("section.post_title")
        title = title_section.get_text(" ", strip=True) if title_section else ""

        # -------- LẤY NỘI DUNG (content) --------
        # chọn tất cả section elementor-element KHÔNG phải post_title
        content_sections = soup.select("#content > div > div > div > section.elementor-element.elementor-element-66303dac.elementor-section-boxed.elementor-section-height-default.elementor-section-height-default.elementor-section.elementor-top-section > div > div > div.elementor-element.elementor-element-3f1e94a8.elementor-column.elementor-col-50.elementor-top-column > div > div > div.elementor-element.elementor-element-37c000cf.elementor-widget.elementor-widget-theme-post-content > div")

        content_text = ""
        for sec in content_sections:
            t = sec.get_text(" ", strip=True)
            if len(t) > 100:  # bỏ qua các section ngắn (vd: quảng cáo)
                content_text += "\n" + t

        text = content_text.strip()

        if not text:
            print(f"[WARN] Không lấy được nội dung từ {url}")
        return text

    except Exception as e:
        print(f"[ERROR] Lỗi khi lấy bài {url}: {e}")
        return ""


# ----------------------------
# Main crawl routine
# ----------------------------
def crawl_viettan_pages(pages):
    driver = create_driver(headless=HEADLESS)
    collected = []
    try:
        for page in pages:
            list_url = f"https://viettan.org/thoi-su/page/{page}/"
            print(f"\n[INFO] Loading listing: {list_url}")
            driver.get(list_url)
            time.sleep(1.5)
            html = driver.page_source

            # Kiểm tra CAPTCHA
            if has_captcha(html, driver=driver):
                print("[ACTION] CAPTCHA detected — please solve it in browser, then press Enter.")
                input("Press Enter after solving CAPTCHA...")

            entries = parse_viettan_listing(html)
            print(f"[INFO] Found {len(entries)} entries on page {page}")

            for title, link in entries:
                if not link:
                    continue
                text = fetch_article_text_selenium(driver, link)
                time.sleep(random.uniform(1.0, 2.0))
                if text and len(text) >= MIN_ARTICLE_LEN:
                    collected.append({
                        "title": title,
                        "text": text,
                        "url": link,
                        "source": "viettan.org",
                        "label": "fake"
                    })

    finally:
        driver.quit()

    df = pd.DataFrame(collected)
    if df.empty:
        print("[WARN] No articles collected.")
    else:
        df = df.drop_duplicates(subset=["title", "text"]).reset_index(drop=True)
    return df

# ----------------------------
# Run
# ----------------------------
if __name__ == "__main__":
    print("Start crawling VietTan pages:", PAGES_TO_CRAWL)
    df_viettan = crawl_viettan_pages(PAGES_TO_CRAWL)
    print("Collected articles:", len(df_viettan))
    if len(df_viettan) > 0:
        out_csv = "viet_tan_Fake.csv"
        df_viettan.to_csv(out_csv, index=False, encoding="utf-8-sig")
        print("Saved to", out_csv)
    else:
        print("No data saved.")


Start crawling VietTan pages: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

[INFO] Loading listing: https://viettan.org/thoi-su/page/1/
[ACTION] CAPTCHA detected — please solve it in browser, then press Enter.
[INFO] Found 0 entries on page 1

[INFO] Loading listing: https://viettan.org/thoi-su/page/2/
[ACTION] CAPTCHA detected — please solve it in browser, then press Enter.
[INFO] Found 12 entries on page 2
[WARN] Không lấy được nội dung từ https://viettan.org/tu-hao-la-dang-vien-viet-tan/

[INFO] Loading listing: https://viettan.org/thoi-su/page/3/
[ACTION] CAPTCHA detected — please solve it in browser, then press Enter.
[INFO] Found 12 entries on page 3

[INFO] Loading listing: https://viettan.org/thoi-su/page/4/
[ACTION] CAPTCHA detected — please solve it in browser, then press Enter.
[INFO] Found 12 entries on page 4

[INFO] Loading lis